# Sayfer AI dasturiga xush kelibsiz! 

- Jarvis dasturiga qiziqadigan odamlar anchagina, yoshlikdan shu sohada biror yangilik yoki dastur chiqmasin, barchasini sinab ko'rish, foydalanishga xohishim baland bo'lgani sabab, O'zbek tilida ham shunday imkoniyatni, shunday dasturiy yengillikni yaratishga maqsad qilganman. 

- Albatta ishlatilgan kodlarni barchasi maniki dimeman, ba'zilari official docs dan olingan, ba'zilari stackoverflow dan olingan, va ba'zilarini esa pythonni o'rganib, o'zim 0 dan tuzib chiqganman, maqsad to'liq O'zbek tilidan ishlaydigan virtual yordamchi yaratish ekan, ortiqcha so'zlarsiz proyektdan foyda ko'rganlar imkon bo'lsa duoda eslab qo'yilar:). -- -- Hammaga rahmat!




## 0) Initialization (kerakli librarylarni yuklash)




In [68]:
import numpy as np
from nltk.stem.porter import PorterStemmer
import torch, string, random, json, subprocess, wikipedia, os, requests, threading, uuid, nltk, openai
from subprocess import call, Popen
import azure.cognitiveservices.speech as speechsdk
from playsound import playsound
from tkinter import *
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from datetime import datetime
from transformers import pipeline
from transformers import RobertaTokenizer, RobertaForQuestionAnswering

mydate = datetime.today().strftime("%y-%m-%d")

key_file = open('../key')

key_data = json.load(key_file)
resource_key = key_data['resource_key']
region = key_data['region']
endpoint = key_data['endpoint']
path = key_data['path']
KEY_FILE = key_data['key_file']
LOCATION = key_data['location']
openai.api_key = key_data['openapi_key']

key_file.close()


start_sequence = "\nAI:"
restart_sequence = "\nHuman: "

session_prompt = "The following is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly.\n\nHuman: Hello, who are you?\nAI: I am an AI created by OpenAI. How can I help you today?\nHuman: "

speech_key, service_region = KEY_FILE, LOCATION
speech_config_stt = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
speech_config_stt.speech_recognition_language="uz-UZ"
#audio_config_stt = speechsdk.audio.AudioConfig(device_name="{0.0.1.00000000}.{c91342cf-e781-4464-93ef-0a8e17804c64}") #headset
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config_stt)

audio_config_tts = speechsdk.audio.AudioConfig(device_name='c91342cf-e781-4464-93ef-0a8e17804c64') #computer
speech_config_tts = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
speech_config_tts.speech_synthesis_voice_name = "uz-UZ-MadinaNeural"
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config_tts)

conf_file = open('assets/settings.conf')
conf_data = json.load(conf_file)

voice_activation = conf_data['voice_activation']
robot_name = conf_data['robot_name'].lower()
gpt3 = conf_data['gpt3']
robertaqna_settings = conf_data['robertaqna']
conf_file.close()

    
wikipedia.set_lang("uz")

#Initialization of robertaqna model parameters

categories = ['sports', 'science', 'technology', 'politics', 'business', 'society']

sport = open('assets/sports-knowledge.json')
sport_knowledge = json.load(sport)
sport_news = sport_knowledge['news']
sport.close()

technology = open('assets/technology-knowledge.json')
technology_knowledge = json.load(technology)
technology_news = technology_knowledge['news']
technology.close()

politics = open('assets/politics-knowledge.json')
politics_knowledge = json.load(politics)
politics_news = politics_knowledge['news']
politics.close()

society = open('assets/society-knowledge.json')
society_knowledge = json.load(society)
society_news = society_knowledge['news']
society.close()

culture = open('assets/culture-knowledge.json')
culture_knowledge = json.load(culture)
culture_news = culture_knowledge['news']
culture.close()

business = open('assets/business-knowledge.json')
business_knowledge = json.load(business)
business_news = business_knowledge['news']
business.close()

factual = open('assets/factual-knowledge.json')
factual_knowledge = json.load(factual)
factual_knowledge_all = factual_knowledge['all']
factual.close()

analytical = open('assets/analytical-knowledge.json')
analytical_knowledge = json.load(analytical)
analytical_knowledge_all = analytical_knowledge['all']
analytical.close()

subjective = open('assets/subjective-knowledge.json')
subjective_knowledge = json.load(subjective)
subjective_knowledge_all = subjective_knowledge['all']
subjective.close()

objective = open('assets/objective-knowledge.json')
objective_knowledge = json.load(objective)
objective_knowledge_all = objective_knowledge['all']
objective.close()




## 1) Birinchi NLTK library dan foydalanib ishlatiladigan funksyilarni ishga tushiramiz.




In [48]:
stemmer = PorterStemmer()

def tokenize(sentence):
	return nltk.word_tokenize(sentence)

def stem(word):
	return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, all_words):
	tokenized_sentence = [stem(w) for w in tokenized_sentence]

	bag = np.zeros(len(all_words), dtype=np.float32)
	for idx, w in enumerate(all_words):
		if w in tokenized_sentence:
			bag[idx] = 1.0

	return bag




## 2) NLTK Utils to'liq ishlashi uchun punkt modeli kerak bo'ladi, endi uni yuklab olishimiz kerak.




In [49]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Peter\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True




## 3) Uchinchi qilgan ishimiz, pytorch orqali modelni yaratish, va uni yuklash




In [50]:
class NeuralNet(nn.Module):
	def __init__(self, input_size, hidden_size, num_classes):
		super(NeuralNet, self).__init__()
		self.l1 = nn.Linear(input_size, hidden_size)
		self.l2 = nn.Linear(hidden_size, hidden_size)
		self.l3 = nn.Linear(hidden_size, num_classes)
		self.relu = nn.ReLU()

	def forward(self, x):
		out = self.l1(x)
		out = self.relu(out)
		out = self.l2(out)
		out = self.relu(out)
		out = self.l3(out)

		#no activation no softmax

		return out




## 4) Endi novbat, modelni intents.json fayl imiz orqali shug'ullantirish (training)




In [51]:
with open('assets/intents.json') as f:
	intents = json.load(f)

all_words = []
tags = []
xy = []

for intent in intents['intents']:
	tag = intent['tag']
	tags.append(tag)
	for pattern in intent['patterns']:
		w = tokenize(pattern)
		all_words.extend(w)
		xy.append((w, tag))

ignore_words = ['?', '!', ',', '.']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

x_train = []
y_train = []

for (pattern_sentence, tag) in xy:
	bag = bag_of_words(pattern_sentence, all_words)
	x_train.append(bag)

	label = tags.index(tag)
	y_train.append(label)

x_train = np.array(x_train)
y_train = np.array(y_train)


class ChatDataset(Dataset):
	def __init__(self):
		self.n_samples = len(x_train)
		self.x_data = x_train
		self.y_data = y_train

	def __getitem__(self, index):
		return self.x_data[index], self.y_data[index]

	def __len__(self):
		return self.n_samples


batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(x_train[0])
learning_rate = 0.001
num_epochs = 1000

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

date = datetime.now().strftime('%d-%m-%Y-%H-%M-%S')

FILE = f"model/data-{date}.pth"

torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

Epoch [100/1000], Loss: 0.7232
Epoch [200/1000], Loss: 0.0603
Epoch [300/1000], Loss: 0.0080
Epoch [400/1000], Loss: 0.0044
Epoch [500/1000], Loss: 0.0023
Epoch [600/1000], Loss: 0.0003
Epoch [700/1000], Loss: 0.0012
Epoch [800/1000], Loss: 0.0005
Epoch [900/1000], Loss: 0.0005
Epoch [1000/1000], Loss: 0.0000
final loss: 0.0000
training complete. file saved to model/data-20-07-2022-15-54-36.pth





## 5) Modeldan javob olish




In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('assets/intents.json', 'r') as f:
    intents = json.load(f)

FILE = "model/data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]


model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = robot_name

def get_response(msg):

    sentence = tokenize(msg)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                answer = random.choice(intent['responses'])
                return answer
    
    return ""





## 6) Savol va Javoblarni Ingliz tilidan Uzbek tiliga va Uzbek tilidan Ingliz tiliga tarjima qilish funksiyalari




In [53]:
def english_to_uzbek(text):
    params = '&from=en&to=uz'
    constructed_url = endpoint + path + params

    headers = {
        'Ocp-Apim-Subscription-Key': resource_key,
        'Ocp-Apim-Subscription-Region': region,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    body = [{
        'text' : str(text)
    }]
    request = requests.post(constructed_url, headers=headers, json=body)
    response = request.json()
    ans = json.loads(json.dumps(response, sort_keys=True, indent=4, separators=(',', ': ')))


    return ans[0]['translations'][0]['text']

def uzbek_to_english(text):
    params = '&from=uz&to=en'
    constructed_url = endpoint + path + params

    headers = {
        'Ocp-Apim-Subscription-Key': resource_key,
        'Ocp-Apim-Subscription-Region': region,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    body = [{
        'text' : str(text)
    }]
    request = requests.post(constructed_url, headers=headers, json=body)
    response = request.json()
    ans = json.loads(json.dumps(response, sort_keys=True, indent=4, separators=(',', ': ')))


    return ans[0]['translations'][0]['text'] 






## 7) Ortiqcha belgilarni textdan olib tashlash




In [54]:
def drop_characters(text):
    a = text
    b = "!@#$()-_=+%&.,\" "
    
    for char in b:
        a = a.replace(char, " ")    

    return a




## 8) Kunlar va yillar qatnashgan tekstlarni ovozli eshittirishga moslashtirish




In [55]:
day_first = {
    1 : "O'n",
    2 : "Yigirma",
    3 : "O'ttiz"
}

day_second = {
    1 : "Birinchi",
    2 : "Ikkinchi",
    3 : "Uchinchi",
    4 : "To'rtinchi",
    5 : "Beshinchi",
    6 : "Oltinchi",
    7 : "Yettinchi",
    8 : "Sakkizinchi",
    9 : "To'qqizinchi",
    0 : "inchi"
}


def day_filter(answer):
    answer = drop_characters(answer)

    digits = [int(s) for s in answer.split() if s.isdigit() and int(s) <= 31 and int(s) >= 10]
    digits2 = [int(s) for s in answer.split() if s.isdigit() and int(s) <= 9]

    digits_in_text = []
    digits_in_text2 = []

    for digit in digits:
        sp = [int(a) for a in str(digit)]
        digits_in_text.append(sp)

    for digit in digits2:
        sp = [int(a) for a in str(digit)]
        digits_in_text2.append(sp)

    day_first_values = []
    day_second_values = []

    day_first_values2 = []

    for p in digits_in_text:
        day_first_values.append(str(p[0]).replace(str(p[0]), day_first[p[0]]))
        day_second_values.append(str(p[1]).replace(str(p[1]), day_second[p[1]]))


    for p in digits_in_text2:
        day_first_values2.append(str(p[0]).replace(str(p[0]), day_second[p[0]]))

    joint_days = {}
    joint_days2 = {}

    for t in range(0, len(digits)):
        days_in_text = day_first_values[t] + " " + day_second_values[t]
        joint_days[digits[t]] = days_in_text 

    for t in range(0, len(digits2)):
        days_in_text2 = day_first_values2[t]
        joint_days2[digits2[t]] = days_in_text2 

    for _ in range(0, len(digits)):
        for t in answer.split():
            if(t.isdigit() and int(t) <= 31 and int(t) >= 10):
                answer = answer.replace(t, joint_days[int(t)])

    for _ in range(0, len(digits2)):
        for t in answer.split():
            if(t.isdigit() and int(t) <= 9):
                answer = answer.replace(t, joint_days2[int(t)])

    return answer

def year_filter(answer):
    year_first = {
        1 : "bir ming",
        2 : "ikki ming",
        3 : "uch ming",
        4 : "to'rt ming",
        5 : "besh ming",
        6 : "olti ming",
        7 : "yetti ming",
        8 : "sakkiz ming",
        9 : "to'qqiz ming"
    }

    year_second = {
        1 : "bir yuz",
        2 : "ikkiyuz",
        3 : "uchyuz",
        4 : "to'rtyuz",
        5 : "beshyuz",
        6 : "oltiyuz",
        7 : "yettiyuz",
        8 : "sakkizyuz",
        9 : "to'qqizyuz",
        0 : ""
    }

    year_third = {
        1 : "o'n",
        2 : "yigirma",
        3 : "o'ttiz",
        4 : "qirq",
        5 : "ellik",
        6 : "oltmish",
        7 : "yetmish",
        8 : "sakson",
        9 : "to'qson",
        0 : ""

    }

    year_fourth = {
        1 : "birinchi",
        2 : "ikkinchi",
        3 : "uchinchi",
        4 : "to'rtinchi",
        5 : "beshinchi",
        6 : "oltinchi",
        7 : "yettinchi",
        8 : "sakkizinchi",
        9 : "to'qqizzinchi",
        0 : "inchi"
    }

    answer = drop_characters(answer)

    digits = [int(s) for s in answer.split() if s.isdigit() and int(s) >= 1000]
    
    digits_in_text = []
    for k in digits:
        seperate_year_digits = [int(b) for b in str(k)]
        digits_in_text.append(seperate_year_digits)

    year_first_values = []
    year_second_values = []
    year_third_values = []
    year_fourth_values = []


    for p in digits_in_text:
        year_first_values.append(str(p[0]).replace(str(p[0]), year_first[p[0]]))
        year_second_values.append(str(p[1]).replace(str(p[1]), year_second[p[1]]))
        year_third_values.append(str(p[2]).replace(str(p[2]), year_third[p[2]]))
        year_fourth_values.append(str(p[3]).replace(str(p[3]), year_fourth[p[3]]))

    joint_years = {}
    for t in range(0, len(digits)):
        years_in_text = year_first_values[t] + " " + year_second_values[t] + " " + year_third_values[t] + " " + year_fourth_values[t]
        joint_years[digits[t]] = years_in_text 


    for _ in range(0, len(digits)):
        for t in answer.split():
            if(t.isdigit() and int(t) >= 1000):
                answer = answer.replace(t, joint_years[int(t)])

    return answer






## 9) Ayni vaqtni aniqlash funksiyasi




In [56]:
def get_current_time():
    answer = datetime.now().strftime("%H-%M").split("-",1)
    hour = int(answer[0])
    minute = int(answer[1])

    if (minute > 9):
        minute_digits = [int(a) for a in str(minute)]
        minute_last = minute_digits[1]
        minute_first = minute_digits[0]

    minute_say = ""

    if minute in range(10, 20):
        minute_say = "o'n"
    elif minute in range(20, 30):
        minute_say = "yigirma"
    elif minute in range(30, 40):
        minute_say = "o'ttiz"
    elif minute in range(40, 50):
        minute_say = "qirq"
    elif minute in range(50, 60):
        minute_say = "ellik"



    if (hour > 9):
        hour_digits = [int(a) for a in str(hour)]
        hour_last = hour_digits[1]
        hour_first = hour_digits[0]

    hour_say = ""

    if hour in range(10, 20):
        hour_say = "o'n"
    elif hour in range(20, 24):
        hour_say = "yigirma"



    if(minute <= 9 and hour <= 9):
        currentTime = f"Soat {hour} dan, {minute} daqiqa o'tdi."
    elif(minute >= 10 and hour <= 9):
        currentTime = f"Soat {hour} dan, {minute_say} {minute_last} daqiqa o'tdi."
    elif(minute <= 9 and hour >= 10):
        currentTime = f"Soat {hour_say} {hour_last} dan, {minute} daqiqa o'tdi."
    elif(minute >= 10 and hour >= 10 and hour_last != 0):
        currentTime = f"Soat {hour_say} {hour_last} dan, {minute_say} {minute_last} daqiqa o'tdi."
    elif(minute >= 10 and hour >= 10 and hour_last == 0):
        currentTime = f"Soat {hour_say} dan, {minute} daqiqa o'tdi."
    elif(minute >= 10 and hour >= 10 and hour_last == 0 and minute_last == 0):
        currentTime = f"Soat {hour_say} dan, {minute} daqiqa o'tdi."
        
    return currentTime

In [57]:
def drop_word(text, word):
    words = text.split()
    for w in words:
        if w == word:
            words.remove(w)
    answer = ' '.join(words)
    return answer

## 10) Tekst klassifikatsiyasi modelini yuklash (text classification model from huggingface)

In [58]:
text_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", cache_dir=".cache/huggingface/transformers/")

In [59]:
def classify_text(query):
    labels = ["artifacts", "animals", "food", "birds", "fast food", "book", "education", "news", "failure", "emotion", "sad",
             "happy", "time", "family", "math", "computer", "general", "language", "drinks", "alcohol", "religion", "Islam",
             "cars"]
    hypothesis_template = "This text is about {} ."
    prediction = text_classifier(query, labels, hypothesis_template=hypothesis_template, multi_label=True)

    labels = prediction['labels'][:3]
    scores = prediction['scores'][:3]
    labels_selected = []
    scores_selected = []
    final_answer = []
    #Qaytarilgan javoblardan 3 ta 75 dan balandlarini tanlab olish
    for score in scores:
        if score >= 0.75:
            scores_selected.append(score)
# Eng yaxshi natijani aniqlash
    if len(scores_selected) == 1:
        labels_selected.append(labels[0])
    elif len(scores_selected) == 2:
        labels_selected.append(labels[0])
        labels_selected.append(labels[1])
    elif len(scores_selected) == 3:
        labels_selected.append(labels[0])
        labels_selected.append(labels[1])
        labels_selected.append(labels[2])
    
    if len(scores_selected) != 0:
        final_answer.append(scores_selected[0])
        final_answer.append(labels_selected[0])
    else:
        final_answer = "No response"
    return final_answer


### 11) Savollarga javob berish va o'rganish modelini yuklash ( huggingface, roberta-base-squad2 )

In [60]:
roberta_qna_tokenizer = RobertaTokenizer.from_pretrained("deepset/roberta-base-squad2", cache_dir=".cache/huggingface/transformers/")
roberta_qna_model = RobertaForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2", cache_dir=".cache/huggingface/transformers/")

In [61]:
def robertaqna(question, text):
    inputs = roberta_qna_tokenizer(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = roberta_qna_model(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    answer = roberta_qna_tokenizer.decode(predict_answer_tokens)
            
    if answer == "<s>":
        answer = "javob topilmadi"
    return answer




## SAYFER AI ni ishga tushirish funksiyasi (Voice activation = 0)




In [70]:
def run_with_no_voice_activation():
    query = drop_characters(speech_recognizer.recognize_once_async().get().text.lower())
    answer = day_filter(year_filter(get_response(query)))
    print(query)

    #Wikipedia knowledge base inclusion (Wikipedia dan qidirish)
    if "haqida" in query and robot_name in query:
        try:
            
            wiki_question = drop_word(query, robot_name).split(' haqida', 1)[0]
            suggested_wiki_answer = wikipedia.suggest(f"{wiki_question}")             

            # if (suggested_wiki_answer is not None):
            #     wiki_answer = random.choice(suggested_wiki_answer.options)
            try:
                speech_synthesizer.speak_text_async(f"{wiki_question} haqida qidiryabman").get()
                wiki_answer = wikipedia.summary(wiki_question, sentences=3)
            except wikipedia.DisambiguationError as e:
                s = e.options[-1]
                wiki_answer = wikipedia.summary(s, sentences=3)

            speech_synthesizer.speak_text_async(day_filter(year_filter(wiki_answer))).get()
        except Exception as e:
            print(e)


    #Dasturlarni ochish
    if("Xrom" in answer and robot_name in query):
        speech_synthesizer.speak_text_async(answer)
        Popen("c:\\Program Files\\Google\\Chrome\\Application\\chrome.exe")
        
 
    elif("Kalkulyator" in answer and robot_name in query):
        speech_synthesizer.speak_text_async(answer)
        call(["calc.exe"])
        

    elif("Telegram" in answer and robot_name in query):
        speech_synthesizer.speak_text_async(answer)
        subprocess.Popen("c:\\Users\\Peter\\AppData\\Roaming\\Telegram Desktop\\Telegram.exe")

    #Soatni so'rash
    if "soat" in query and robot_name in query.split():
        currentTime = get_current_time()
        speech_synthesizer.speak_text_async(currentTime).get()
    
    #TakeNotes malumotlarni saqlab qolish

#     if "eslab qol" in query and "sayfer" in query.split():
#         note = query.split("eslab qol")[0]
#         print(note)


#         k = open('knowledgeNotes.json', 'w')
#         k.write(json.dumps(knowledgeNotes))
#         k.close()

#         g = open('generalNotes.json', 'w')
#         g.write(json.dumps(generalNotes))
#         g.close()
#         return 1


#     #ReadNotes Ma'lumotlarni o'qish
#     if answer == "Bugungi kiritilgan ilmiy ma'lumotlarni o'qib eshittiraman":
#         if(os.path.isfile('knowledgeNotes.json')):
#             knowledgeNotesRaw = open('knowledgeNotes.json')
#             knowledgeNotes = json.load(knowledgeNotesRaw)
#             knowledgeNotesRaw.close()
#         else:
#             speech_synthesizer.speak_text_async("Ilmiy ma'lumotlar bazasi topilmadi")

#         knowledgeNotesText = []
#         knowledgeNotesDate = []


#         for q in knowledgeNotes["dates"]:
#             date = datetime.strptime(q, '%y-%m-%d')
#             date_final = date.strftime('%y-%m-%d')
#             knowledgeNotesDate.append(date_final)

#         for i in knowledgeNotes["notes"]:
#             knowledgeNotesText.append(i)

#         knowledgeNotesArray = np.stack((knowledgeNotesText, knowledgeNotesDate), axis=1)

#         speakingKnowledgeNotes = []

#         for data in knowledgeNotesArray:
#             if data[1] == mydate:
#                 speakingKnowledgeNotes.append(data[0])


#         if len(speakingKnowledgeNotes) == 0:
#             speech_synthesizer.speak_text_async("Bugungi kiritilgan ilmiy ma'lumotlar mavjud emas")
#         else:
#             speech_synthesizer.speak_text_async("Bugungi kiritilgan ilmiy ma'lumotlarni o'qib eshittiraman")
#             for note in speakingKnowledgeNotes:
#                 speech_synthesizer.speak_text_async(note)
#         return 1

#         for k in generalNotes["dates"]:
#             date = datetime.strptime(k, '%y-%m-%d')
#             date_final = date.strftime('%y-%m-%d')
#             generalNotesDate.append(date_final)

#         for t in generalNotes["notes"]:
#             generalNotesText.append(t)

#         generalNotesArray = np.stack((generalNotesText, generalNotesDate), axis=1)

#         speakingGeneralNotes = []

#         for data in generalNotesArray:
#             if data[1] == mydate:
#                 speakingGeneralNotes.append(data[0])

#         if len(speakingGeneralNotes) == 0:
#             speech_synthesizer.speak_text_async("Bugungi kiritilgan umumiy ma'lumotlar mavjud emas")
#         else:
#             speech_synthesizer.speak_text_async("Bugungi kiritilgan umumiy ma'lumotlarni o'qib eshittiraman")
#             for note in speakingGeneralNotes:
#                 speech_synthesizer.speak_text_async(note)




## 11) SAYFER AI ni ishga tushirish funksiyasi (Voice activation = 1)




In [63]:
def run_with_voice_activation():
    sayfer_status = "offline" 

    if(sayfer_status == "offline"):
        query = speech_recognizer.recognize_once_async().get().text
        print(query)
        if(robot_name in query.lower()):
            sayfer_status = "online"

    if(sayfer_status == "online"):

        playsound('assets\\audio.mp3', block=False)
        query = speech_recognizer.recognize_once_async().get().text
        answer = day_filter(year_filter(get_response(query)))
        print(query)

        get_response(query)

        #Wikipedia knowledge base inclusion (Wikipedia dan qidirish)
        if ("haqida" in query):
            sayfer_status = "offline"
            try:
                wiki_question = query.split(' haqida', 1)[0]
                suggested_wiki_answer = wikipedia.suggest(f"{wiki_question}")             

                try:
                    speech_synthesizer.speak_text_async(f"{wiki_question} haqida qidiryabman").get()
                    wiki_answer = wikipedia.summary(wiki_question, sentences=3)
                except wikipedia.DisambiguationError as e:
                    s = e.options[-1]
                    wiki_answer = wikipedia.summary(s, sentences=3)

                speech_synthesizer.speak_text_async(day_filter(year_filter(wiki_answer))).get()

            except Exception as e:
                print(e)

        #Dasturlarni ochish
        if("Xrom" in answer):
            call(["chrome.exe"])
            sayfer_status = "offline"

        elif("Kalkulyator" in answer):
            call(["calc.exe"])
            sayfer_status = "offline"

        elif("Telegram" in answer):
            subprocess.Popen("c:\\Users\\Peter\\AppData\\Roaming\\Telegram Desktop\\Telegram.exe")
            sayfer_status = "offline"


        #Soatni so'rash
        sayfer_status = "online"
        if ("soat" in query.lower()):
            currentTime = get_current_time()
            speech_synthesizer.speak_text_async(currentTime).get()
            sayfer_status = "offline"


        if(sayfer_status != "offline"):
            #Buyruq aniqlanmagan holatda
            if(answer == "tushunmadim"):
                sayfer_status = "offline"
                speech_synthesizer.speak_text_async("tushunmadim").get()
            else:
                answer = day_filter(year_filter(answer))
                speech_synthesizer.speak_text_async(answer).get()





## GPT-3 ni ishga tushirish funksiyasi




In [64]:
def run_gpt3(question, chat_log=None):
  global session_prompt
  if chat_log is None: 
    chat_log = session_prompt 

  prompt_text = f'{chat_log}{restart_sequence}: {question}{start_sequence}:'
  
  response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=prompt_text,
    temperature=0.9,
    max_tokens=150,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0.6,
    stop=[" Human:", " AI:"]
    #stop=["\n"],
  )
  story = response['choices'][0].text

  session_prompt = f'{chat_log}{restart_sequence}: {question}{start_sequence}: {story}:'
  return str(story)




## Umumiy dasturni ishga tushirish




In [71]:
if __name__ == "__main__":
    while True:
        if voice_activation == "False":
            run_with_no_voice_activation()
        elif voice_activation == "True":
            run_with_voice_activation()
        
        if gpt3 == "True":
            activation = speech_recognizer.recognize_once_async().get().text.lower()
            if(robot_name in activation):
                playsound('assets/audio.mp3', block=False)
                qinUz = speech_recognizer.recognize_once_async().get().text.lower()
                qinEn = uzbek_to_english(qinUz)
                aninEn = run_gpt3(qinEn)
                aninUz = english_to_uzbek(aninEn)
                print(qinUz)
                print(aninUz)
                speech_synthesizer.speak_text_async(aninUz).get()
                
        if robertaqna_settings == "True":
            activation = speech_recognizer.recognize_once_async().get().text.lower()
            if(robot_name in activation):
                playsound('assets/audio.mp3', block=False)
                qinUz = speech_recognizer.recognize_once_async().get().text.lower()
                qinEn = uzbek_to_english(qinUz)
                aninEn = robertaqna(qinEn, technology_news)
                aninUz = english_to_uzbek(aninEn)
                print(qinUz)
                print(aninUz)
                speech_synthesizer.speak_text_async(year_filter(aninUz)).get()

sayfer soatni aytib yubor 
sayfer soat 
soatda aytib yubor 
i 
sayfer soat necha bo'ldi 
sayfer kalkulyatorni ochib berdi 

sayfer telegraf doch ber 
i sayfer telegram'ni ochib berdi 


i 


SystemError: <built-in function SpeechRecognitionResultPtrFuture_get> returned a result with an exception set